In [15]:
import sqlite3
import numpy as np
import pandas as pd
import time
from IPython.display import display

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('L19DB_demo.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = []
    for i, col_name in enumerate(cols):
        framelist.append((col_name, [col[i] for col in q]))
    return pd.DataFrame.from_items(framelist)

query = '''SELECT * FROM contributors WHERE candidate_id = (SELECT id from candidates WHERE last_name = "Obama")'''
display(viz_tables(contributor_cols, query))

query = '''SELECT * FROM candidates'''
display(viz_tables(candidate_cols, query))

# INNER JOIN
# 1
query = '''SELECT contributors.id, contributors.last_name, contributors.first_name, candidates.last_name FROM candidates INNER JOIN contributors ON candidates.id = contributors.candidate_id'''
display(viz_tables(['contributors.id', 'contributors.last_name', 'contributors.first_name', 'candidates.last_name'], query))

# 2
query = '''SELECT contributors.id, contributors.last_name, contributors.first_name, candidates.last_name FROM candidates INNER JOIN contributors ON candidates.id = contributors.candidate_id WHERE candidates.last_name = "Obama"'''
display(viz_tables(['contributors.id', 'contributors.last_name', 'contributors.first_name', 'candidates.last_name'], query))

# LEFT JOIN
# 3
query = '''SELECT AVG(contributors.amount), COUNT(contributors.id), candidates.last_name FROM candidates LEFT JOIN contributors ON candidates.id = contributors.candidate_id GROUP BY candidates.id'''
display(viz_tables(['average contribution', 'number of contributors', 'candidate last name'], query))
db.close()

# Pandas
dfcand = pd.read_csv("candidates.txt", sep="|")
dfcontr = pd.read_csv("contributors.txt", sep="|")
del dfcontr['id']

dbp = sqlite3.connect('L19_pandas_DB.sqlite')
csr = dbp.cursor()
csr.execute("DROP TABLE IF EXISTS candidates")
csr.execute("DROP TABLE IF EXISTS contributors")
csr.execute("PRAGMA foreign_keys=1")

csr.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_name TEXT, 
               party TEXT NOT NULL)''')

dbp.commit() # Commit changes to the database

csr.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

dbp.commit()

dfcand.to_sql("candidates", dbp, if_exists="append", index=False)

# 1
dfcontr.to_sql('contributors', dbp, if_exists='append', index=False)
display(dfcontr.head())
print(dfcontr.shape)

# 2A
display(dfcontr.query('state == "VA" & amount < 400'))

# 2B
display(dfcontr[dfcontr.state.isnull()])

# 2C
display(dfcontr.query('state == "TX" | state == "PA"'))

# 2D
display(dfcontr.query('amount >= 10 & amount <= 50'))

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,26,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,50.00,2007-07-30,20
1,27,Buckler,Steve,,24351 Armada Dr,,Dana Point,CA,926291306,25.00,2007-08-16,20
2,28,Buckheit,Bruce,,8904 KAREN DR,,FAIRFAX,VA,220312731,100.00,2007-09-19,20
3,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.00,2007-08-14,20
4,30,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,-2300.00,2007-08-14,20
5,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.00,2007-08-14,20
6,32,Buck,Thomas,,4206 Terrace Street,,Kansas City,MO,64111,100.00,2007-09-25,20
7,33,Buck,Jay,K.,1855 Old Willow Rd Unit 322,,Northfield,IL,600932918,200.00,2007-09-12,20
8,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.00,2007-09-30,20
9,35,Buck,Barbara,,1780 NE 138th St,,North Miami,FL,331811316,50.00,2007-09-13,20


,id,first_name,last_name,middle_init,party
0,16,Mike,Huckabee,,R
1,20,Barack,Obama,,D
2,22,Rudolph,Giuliani,,R
3,24,Mike,Gravel,,D
4,26,John,Edwards,,D
5,29,Bill,Richardson,,D
6,30,Duncan,Hunter,,R
7,31,Dennis,Kucinich,,D
8,32,Ron,Paul,,R
9,33,Joseph,Biden,,D


,contributors.id,contributors.last_name,contributors.first_name,candidates.last_name
0,1,Agee,Steven,Huckabee
1,2,Ahrens,Don,Huckabee
2,3,Ahrens,Don,Huckabee
3,4,Ahrens,Don,Huckabee
4,5,Akin,Charles,Huckabee
5,6,Akin,Mike,Huckabee
6,7,Akin,Rebecca,Huckabee
7,8,Aldridge,Brittni,Huckabee
8,9,Allen,John D.,Huckabee
9,10,Allen,John D.,Huckabee


,contributors.id,contributors.last_name,contributors.first_name,candidates.last_name
0,26,Buckler,Steve,Obama
1,27,Buckler,Steve,Obama
2,28,Buckheit,Bruce,Obama
3,29,Buckel,Linda,Obama
4,30,Buckel,Linda,Obama
5,31,Buckel,Linda,Obama
6,32,Buck,Thomas,Obama
7,33,Buck,Jay,Obama
8,34,Buck,Blaine,Obama
9,35,Buck,Barbara,Obama


,average contribution,number of contributors,candidate last name
0,772.0000,25,Huckabee
1,472.4356,25,Obama
2,-1360.0000,25,Giuliani
3,NaN,0,Gravel
4,NaN,0,Edwards
5,NaN,0,Richardson
6,NaN,0,Hunter
7,NaN,0,Kucinich
8,-519.7088,25,Paul
9,NaN,0,Biden


,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


(175, 11)


,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
125,BOURNE,TRAVIS,NaN,LAGE KAART 77,NaN,BRASSCHATT,NaN,2930,-500.0,2008-11-20,35


,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
65,BURGERT,RONALD,L.,5723 PLUMTREE DRIVE,NaN,DALLAS,TX,752524926,-1000.00,2008-03-05,22
70,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.00,2008-03-05,22
71,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.00,2008-03-05,22
79,Muse,Nina,Jo,2915 Toro Canyon Rd,NaN,Austin,TX,78746,-50.00,2008-04-21,32
93,Raught,Philip,M,4714 Plum Way,NaN,Pittsburgh,PA,15201,-1046.00,2008-04-21,32
94,Ferrara,Judith,D,1508 Waterford Road,NaN,Yardley,PA,19067,-1100.00,2008-04-21,32
95,Johnson,Cathleen,E.,1003 Justin Ln Apt 2016,NaN,Austin,TX,787572648,-14.76,2008-04-21,32
132,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,50.00,2007-09-30,35
133,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,150.00,2007-08-09,35
134,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,50.00,2007-07-19,35


### LEFT JOIN
#### Exercise 1
| nA | A.attr  | nB  | B.attr  |
|---|---|---|---|
| s1  | 23  | t1  | 60  |
| s2  | 7  | t3  | 22  |
| s3  | 15  | t3  | 22  |
| s4  | 31  | t1 | 60 |

#### Exercise 2
|nB|B.attr|nA|A.attr|
|---|---|---|---|
|t1|60|s1|23|
|t1|60|s4|0|
|t2|14|NaN|NaN|
|t3|22|s2|7|
|t3|22|s3|15|